In [ ]:
import os

os.chdir("..")

In [ ]:
import torch
from diffusers.pipelines import FluxPipeline
from PIL import Image

from omini.pipeline.flux_omini import Condition, generate, seed_everything

In [ ]:
pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16
)
pipe = pipe.to("cuda")

In [ ]:
pipe.unload_lora_weights()

for style_type in ["ghibli", "irasutoya", "simpsons", "snoopy"]:
    pipe.load_lora_weights(
        "Yuanshi/OminiControlArt",
        weight_name=f"v0/{style_type}.safetensors",
        adapter_name=style_type,
    )

pipe.set_adapters(["ghibli", "irasutoya", "simpsons", "snoopy"])

In [ ]:
def resize(img, factor=16):
    # Resize the image to be divisible by the factor
    w, h = img.size
    new_w, new_h = w // factor * factor, h // factor * factor
    padding_w, padding_h = (w - new_w) // 2, (h - new_h) // 2
    img = img.crop((padding_w, padding_h, new_w + padding_w, new_h + padding_h))
    return img


def bound_image(image):
    factor = 512 / max(image.size)
    image = resize(
        image.resize(
            (int(image.size[0] * factor), int(image.size[1] * factor)),
            Image.LANCZOS,
        )
    )
    delta = (0, -image.size[0] // 16)
    return image, delta

sizes = {
    "2:3": (640, 960),
    "1:1": (640, 640),
    "3:2": (960, 640),
}

In [ ]:
image = Image.open("assets/ominicontrol_art/DistractedBoyfriend.webp").convert("RGB")
image, delta = bound_image(image)
condition = Condition(image, "ghibli", position_delta=delta)

seed_everything()

size = sizes["3:2"]

result_img = generate(
    pipe,
    prompt="",
    conditions=[condition],
    max_sequence_length=32,
    width=size[0],
    height=size[1],
    image_guidance_scale=1.5,
).images[0]


In [ ]:
image = Image.open("assets/ominicontrol_art/oiiai.png").convert("RGB")
image, delta = bound_image(image)
condition = Condition(image, "irasutoya", position_delta=delta)

seed_everything()

size = sizes["1:1"]

result_img = generate(
    pipe,
    prompt="",
    conditions=[condition],
    max_sequence_length=32,
    width=size[0],
    height=size[1],
    image_guidance_scale=1.5,
).images[0]

result_img

In [ ]:
image = Image.open("assets/ominicontrol_art/breakingbad.jpg").convert("RGB")
image, delta = bound_image(image)
condition = Condition(image, "simpsons", position_delta=delta)

seed_everything()

size = sizes["3:2"]

result_img = generate(
    pipe,
    prompt="",
    conditions=[condition],
    max_sequence_length=32,
    width=size[0],
    height=size[1],
    image_guidance_scale=1.5,
).images[0]

result_img

In [ ]:
image = Image.open("assets/ominicontrol_art/PulpFiction.jpg").convert("RGB")
image, delta = bound_image(image)
condition = Condition(image, "snoopy", position_delta=delta)

seed_everything()

size = sizes["3:2"]

result_img = generate(
    pipe,
    prompt="",
    conditions=[condition],
    max_sequence_length=32,
    width=size[0],
    height=size[1],
    image_guidance_scale=1.5,
).images[0]

result_img